# GROUPING

After preparing the data for a more exhaustive analysis, the last step is to separate the different countries into group with common characteristics. This is necessary as an aggregate study of all countries would mean erasing the differences between them and missing important conclusions, making our study too shallow.

In [4]:
import os
import pandas as pd
column_country = 'Country'
column_region = 'Region'

read_path = os.getcwd()
region_url = read_path + '/Databases/AuxiliarData/world-regions.csv'
df_url = read_path + '/Output/SilverDataframe.csv'

write_path = os.getcwd() + '/Output'
country_folder = write_path + '/Country'
region_folder = write_path + '/Region'

In [5]:
region_df = pd.read_csv(region_url)
df = pd.read_csv(df_url)

print(region_df)
region_df.rename(columns = {'World Region according to the World Bank': column_region, 'Entity': column_country}, inplace = True)
region_df.drop(columns=['Code', 'Year'], inplace = True)
print(region_df)

print(df)
df = pd.merge(df, region_df)
print(df)

if not os.path.exists(write_path):
    os.makedirs(write_path)

df.to_csv(write_path + '/GoldDataframe.csv')

if not os.path.exists(country_folder):
    os.makedirs(country_folder)

if not os.path.exists(region_folder):
    os.makedirs(region_folder)

for country in set(region_df[column_country].to_list()):
    df.loc[df[column_country] == country].to_csv(country_folder + '' + country + '.csv')

for region in set(region_df[column_region].to_list()):
    df.loc[df[column_region] == region].to_csv(region_folder + '' + region + '.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\vperezlo\\Documents\\GitHub\\python-data-driven-decisions/SilverDataframe.csv'

In [ ]:
df.loc[:, 1:2]

TypeError: cannot do slice indexing on Index with these indexers [1] of type int

In [ ]:
# K-MEANS CLUSTERING
# Importing Modules
from sklearn import datasets
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
# Loading dataset
iris_df = datasets.load_iris()

# Declaring Model
model = KMeans(n_clusters=3)

# Fitting Model
model.fit(iris_df.data)

# Predicitng a single input
predicted_label = model.predict([[7.2, 3.5, 0.8, 1.6]])

# Prediction on the entire data
all_predictions = model.predict(iris_df.data)

# Printing Predictions
print(predicted_label)
print(all_predictions)


# import some data to play with
iris = datasets.load_iris()
X = iris.data[:, :3]  # we only take the first two features.
y = iris.target


fig = plt.figure(figsize=(10,10))
plt = fig.add_subplot(111, projection='3d')
plt.scatter(X[:,0],X[:,1],X[:,2], 
            c=all_predictions, edgecolor='red', s=40, alpha = 0.5)
plt.set_title("First three PCA directions")
plt.set_xlabel("Educational_Degree")
plt.set_ylabel("Gross_Monthly_Salary")
plt.set_zlabel("Claim_Rate")
plt.dist = 10
plt

In [ ]:
from sklearn import datasets
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [ ]:
model = KMeans()
res = model.fit_predict(df)

ValueError: could not convert string to float: 'Afghanistan'